In [322]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
from numpy.linalg import norm
from math import *

In [2]:
root_path = ! pwd
root_path = root_path[0] + '/'
root_path

'/home/ruslan/Desktop/Swarm/Swarmskin/swarmskin_data/'

In [188]:
cf_names = ['cf1', 'cf2', 'cf3', 'cf4']
lp_names = ['lp1', 'lp2', 'lp3', 'lp4']

# One Drone experiment 

In [67]:
def err_after_land(PATH, cf_name, vizualize=0):
    cf  = pd.read_csv(PATH + '_slash_vicon_slash_'+cf_name+'_slash_'+cf_name+'.csv')
    lp1 = pd.read_csv(PATH + '_slash_vicon_slash_lp1_slash_lp1.csv')
    lp2 = pd.read_csv(PATH + '_slash_vicon_slash_lp2_slash_lp2.csv')
    lp3 = pd.read_csv(PATH + '_slash_vicon_slash_lp3_slash_lp3.csv')
    lp4 = pd.read_csv(PATH + '_slash_vicon_slash_lp4_slash_lp4.csv')
    
    
    if 'lp1' in PATH: lp = lp1
    if 'lp2' in PATH: lp = lp2
    if 'lp3' in PATH: lp = lp3
    if 'lp4' in PATH: lp = lp4
    
    land_point = np.array([cf.x[len(cf.x)-1], cf.y[len(cf.y)-1]])
    lp_last    = np.array([lp.x[len(cf.x)-1], lp.y[len(cf.y)-1]])
    err = np.linalg.norm(land_point - lp_last)
    
    if vizualize:
        plt.figure()
        plt.plot(cf.x, cf.y, label='drone')
        plt.plot(lp.x, lp.y, label='lp')
        plt.plot( cf.x[len(cf.x)-1], cf.y[len(cf.y)-1], 'ro', markersize=10, label='landing point')
        plt.plot(lp_last[0], lp_last[1], '*')
        plt.legend()
    return err

In [68]:
def metrics(path):
    land_errors = []
    for cf_name in cf_names:
        for folder_name in os.listdir(path):
            try:
                land_errors.append( err_after_land(path+folder_name+'/', cf_name) )
            except:
                pass
    return np.mean(land_errors), np.std(land_errors), np.max(land_errors)

## Errors after landing

### Tactile feedback

In [69]:
path = root_path + '1_drone/tactile/'
metrics(path)

(0.02899167451805656, 0.014098639837329488, 0.05705442775990614)

### Visual feedback

In [70]:
path = root_path + '1_drone/visual/'
metrics(path)

(0.015972032782265257, 0.009801790145905598, 0.046356056637016134)

### Visual + Tactile feedback

In [71]:
path = root_path + '1_drone/visual_tactile/'
metrics(path)

(0.011806312540366437, 0.006911032914843055, 0.033802840498465346)

## Errors during landing

In [349]:
def err_during_land(PATH, cf_file, vizualize=0):
    cf  = pd.read_csv(PATH + '_slash_vicon_slash_'+cf_name+'_slash_'+cf_name+'.csv')
    lp1 = pd.read_csv(PATH + '_slash_vicon_slash_lp1_slash_lp1.csv')
    lp2 = pd.read_csv(PATH + '_slash_vicon_slash_lp2_slash_lp2.csv')
    lp3 = pd.read_csv(PATH + '_slash_vicon_slash_lp3_slash_lp3.csv')
    lp4 = pd.read_csv(PATH + '_slash_vicon_slash_lp4_slash_lp4.csv')
    
    if 'lp1' in PATH: lp = lp1
    if 'lp2' in PATH: lp = lp2
    if 'lp3' in PATH: lp = lp3
    if 'lp4' in PATH: lp = lp4
        
    length = min(len(lp.x), len(cf.x))
    err = np.mean( np.sqrt( np.array(lp.x[:length] - cf.x[:length])**2 + np.array(lp.y[:length] - cf.y[:length])**2 ) )
#     mean(norm(lp - cf))
    
    if vizualize:
        plt.figure()
        plt.plot(cf.x, cf.y, label='drone')
        plt.plot(lp.x, lp.y, label='lp')
        plt.legend()
        
    return err

### Visual feedback

In [351]:
path = root_path + '1_drone/visual/'
follow_errs = []
for cf_name in cf_names:
    for folder_name in os.listdir(path):
        try:
            err = err_during_land(path+folder_name+'/', cf_name, vizualize=0)
            follow_errs.append( err )
        except:
            # print 'no crazyflie data in folder'
            pass
print round(100*np.mean(follow_errs),2), 'cm, ', round(100*np.std(follow_errs),2), 'cm, ', round(100*np.max(follow_errs),2), 'cm '

3.26 cm,  1.22 cm,  6.0 cm 


### Tactile feedback

In [352]:
path = root_path + '1_drone/tactile/'
follow_errs = []
for cf_name in cf_names:
    for folder_name in os.listdir(path):
        try:
            err = err_during_land(path+folder_name+'/', cf_name, vizualize=0)
            if not np.isnan(err): follow_errs.append( err )
        except:
            # print 'no crazyflie data in folder'
            pass
print round(100*np.mean(follow_errs),2), 'cm, ', round(100*np.std(follow_errs),2), 'cm, ', round(100*np.max(follow_errs),2), 'cm '

3.6 cm,  1.33 cm,  6.11 cm 


### Visual + Tactile feedback

In [353]:
path = root_path + '1_drone/visual_tactile/'
follow_errs = []
for cf_name in cf_names:
    for folder_name in os.listdir(path):
        try:
            err = err_during_land(path+folder_name+'/', cf_name, vizualize=0)
            if not np.isnan(err): follow_errs.append( err )
        except:
            # print 'no crazyflie data in folder'
            pass
print round(100*np.mean(follow_errs),2), 'cm, ', round(100*np.std(follow_errs),2), 'cm, ', round(100*np.max(follow_errs),2), 'cm '

2.46 cm,  0.88 cm,  4.75 cm 


# Four Drones experiment

In [266]:
def err_after_land(PATH, vizualize=0):
    cf  = pd.read_csv(PATH + '_slash_vicon_slash_'+cf_name+'_slash_'+cf_name+'.csv')
    lp  = pd.read_csv(PATH + '_slash_vicon_slash_'+lp_name+'_slash_'+lp_name+'.csv')
    
    land_point = np.array([cf.x[len(cf.x)-1], cf.y[len(cf.y)-1]])
    lp_last    = np.array([lp.x[len(lp.x)-1], lp.y[len(lp.y)-1]])
    err = np.linalg.norm(land_point - lp_last)
    
    if vizualize:
        plt.figure()
        plt.plot(cf.x, cf.y, label='drone')
        plt.plot(lp.x, lp.y, label='lp')
        plt.plot( cf.x[len(cf.x)-1], cf.y[len(cf.y)-1], 'ro', markersize=10, label='landing point')
        plt.plot(lp_last[0], lp_last[1], '*')
        plt.title(cf_name+' '+lp_name)
        plt.legend()
    return err

In [340]:
def err_during_land(PATH, vizualize=0):
    cf  = pd.read_csv(PATH + '_slash_vicon_slash_'+cf_name+'_slash_'+cf_name+'.csv')
    lp  = pd.read_csv(PATH + '_slash_vicon_slash_'+lp_name+'_slash_'+lp_name+'.csv')

    length = min(len(lp.x), len(cf.x))
    err = np.mean( np.sqrt( np.array(lp.x[:length] - cf.x[:length])**2 + np.array(lp.y[:length] - cf.y[:length])**2 ))
    
    if vizualize:
        plt.figure()
        plt.plot(cf.x, cf.y, label='drone')
        plt.plot(lp.x, lp.y, label='lp')
        plt.legend()
    return err

## Error after landing

### Visual feedback

In [341]:
path = root_path + '4_drones/visual/'
land_errors = []
for folder_name in os.listdir(path):
    cf_name = folder_name[:3]
    lp_name = folder_name[4:7]
    land_errors.append( err_after_land(path+folder_name+'/', vizualize=0) )
print round(100*np.mean(land_errors),2), 'cm, ', round(100*np.std(land_errors),2), 'cm, ', round(100*np.max(land_errors),2), 'cm '

2.26 cm,  1.34 cm,  5.56 cm 


### Tactile feedback

In [342]:
path = root_path + '4_drones/tactile/'
land_errors = []
for folder_name in os.listdir(path):
    cf_name = folder_name[:3]
    lp_name = folder_name[4:7]
    land_errors.append( err_after_land(path+folder_name+'/', vizualize=0) )
print round(100*np.mean(land_errors),2), 'cm, ', round(100*np.std(land_errors),2), 'cm, ', round(100*np.max(land_errors),2), 'cm '

3.72 cm,  2.46 cm,  10.8 cm 


### Visual + Tactile feedback

In [343]:
path = root_path + '4_drones/visual_tactile/'
land_errors = []
for folder_name in os.listdir(path):
    cf_name = folder_name[:3]
    lp_name = folder_name[4:7]
    land_errors.append( err_after_land(path+folder_name+'/', vizualize=0) )
print round(100*np.mean(land_errors),2), 'cm, ', round(100*np.std(land_errors),2), 'cm, ', round(100*np.max(land_errors),2), 'cm '

2.31 cm,  1.89 cm,  6.7 cm 


## Error during landing

### Visual feedback

In [344]:
path = root_path + '4_drones/visual/'

during_land_errors = []
for folder_name in os.listdir(path):
    cf_name = folder_name[:3]
    lp_name = folder_name[4:7]
    during_land_errors.append( err_during_land(path+folder_name+'/', vizualize=0) )
print round(100*np.mean(during_land_errors),2), 'cm, ', round(100*np.std(during_land_errors),2), 'cm, ', round(100*np.max(during_land_errors),2), 'cm '

3.81 cm,  1.57 cm,  7.8 cm 


### Tactile feedback

In [345]:
path = root_path + '4_drones/tactile/'

during_land_errors = []
for folder_name in os.listdir(path):
    cf_name = folder_name[:3]
    lp_name = folder_name[4:7]
    during_land_errors.append( err_during_land(path+folder_name+'/', vizualize=0) )
print round(100*np.mean(during_land_errors),2), 'cm, ', round(100*np.std(during_land_errors),2), 'cm, ', round(100*np.max(during_land_errors),2), 'cm '

4.8 cm,  3.56 cm,  17.47 cm 


### Visual + Tactile feedback

In [346]:
path = root_path + '4_drones/visual_tactile/'

during_land_errors = []
for folder_name in os.listdir(path):
    cf_name = folder_name[:3]
    lp_name = folder_name[4:7]
    during_land_errors.append( err_during_land(path+folder_name+'/', vizualize=0) )
print round(100*np.mean(during_land_errors),2), 'cm, ', round(100*np.std(during_land_errors),2), 'cm, ', round(100*np.max(during_land_errors),2), 'cm '

3.29 cm,  1.07 cm,  5.65 cm 
